# These following codes are intended for calculating 1st deriative of basepairs over time and then detecting the change-points of 1st deriative (seperate matlab code), thus finding the rate changing points 

In [1]:
# -*-coding:utf-8 -*-
# import all the libraries 
# python==3.8; jupyterlab==3.0.12; lumicks.pylake==0.8.1; matplotlib==3.3.4; more-itertools==8.7.0;
# npTDMS==1.1.0; numpy==1.20.1; opencv-python==4.5.1.48; pandas==1.2.3; scipy==1.6.1; tifffile==2021.3.5
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from nptdms import TdmsFile
from scipy.signal import savgol_filter
import tifffile as tif
from scipy import interpolate
%matplotlib widget

In [71]:
# read basepair-time data and plot it
name = input('please type in the file name:' ) + '.xlsx'
path = input('please type in the file path:')
file = os.path.join(path, name) 
data = pd.read_excel(file)
print(data.head())

please type in the file name: 20200821-017-002-processedData
please type in the file path: C:\Users\KTS260\Google Drive\T7 Helicase data_work_with_Pim\gp4+gp2.5


   Unnamed: 0  time  ssDNA_all_percentage  junction_position_all    basepairs
0           0  3450              0.006872               0.032010  8335.321026
1           1  3500              0.008405               0.039148  8322.460641
2           2  3551              0.009495               0.044226  8313.309863
3           3  3600              0.012396               0.057738  8288.963067
4           4  3650              0.008926               0.041578  8318.081443


In [72]:
# plot basepair changes as a function of time
plt.figure(figsize=(8,6))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.xlabel('Time (s)',fontdict=font)
plt.ylabel('Basepairs',fontdict=font)
plt.plot(data['time']/1000,data['basepairs'],color='red', linestyle='dashed',linewidth=1, label='Basepairs')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
# this step is to find out the optimal window size that savgol_filter will use below, thus to reduce the noise without losing too much information
residuals = []
kernal_size = np.linspace(5,101,49)
for i in kernal_size:
    basepairs_sgfilt = savgol_filter(data['basepairs'],int(i),3)
    residual = np.sum((basepairs_sgfilt-data['basepairs'])**2)
    residuals.append(residual)
print(residuals)
residual_norm = residuals/(np.max(residuals))
plt.figure(figsize = (5,3))
plt.scatter(kernal_size,residual_norm)
# plt.title('Finding the optimal window size')
plt.axhline(0.99)
# plt.xscale('log')
plt.yscale('log')
# plt.xlabel("window size")
# plt.ylabel("normalized residuals")
# plt.ylim(0.8,1.1)
# plt.xlim(0,85)
plt.show()
plt.tight_layout()

[591281.0933243658, 871072.5021164485, 1052640.3270835974, 1195435.6387222153, 1307950.219760824, 1394024.3333459492, 1468958.3607706076, 1535221.5921047537, 1591542.7398892206, 1643661.520255188, 1685030.1126916932, 1732932.0060389726, 1777349.7030811869, 1824288.021248111, 1866876.4861890892, 1903038.1876969263, 1937235.862758499, 1966732.9147818778, 1990350.5574183261, 2011495.5529226884, 2033910.5511330618, 2058711.54029197, 2089023.028585027, 2118757.0195805896, 2145062.991236046, 2173530.2647069218, 2202044.4312472297, 2231598.8903195774, 2263220.564233667, 2294922.8610657547, 2328187.440136683, 2361197.7334868116, 2392223.9118985906, 2421771.153603432, 2451043.9584291815, 2480242.4862208124, 2512264.5329718655, 2541054.0341626615, 2572003.0117540313, 2601143.131653771, 2628634.2305872655, 2656622.4211212653, 2687351.283720374, 2718044.3525266447, 2748868.8827737262, 2779501.775991495, 2809031.42453254, 2837230.291553267, 2863467.2438268936]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
# to plot the filtered data as well as raw data
basepairs_filter = savgol_filter(data['basepairs'], 305, 3)
plt.figure(figsize=(8,6))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.xlabel('Time (s)',fontdict=font)
plt.ylabel('Basepairs',fontdict=font)
plt.plot(data['time']/1000,data['basepairs'],color='red', linestyle='dashed',linewidth=1, label='Basepairs')
plt.plot(data['time']/1000,basepairs_filter,color='black',linewidth=1, label='SG_filter-windsize=65')
# plt.plot(data['time']/1000,basepairs_medfilt,color='yellow',linewidth=1, label='med_filter-kernalsize=27')

# plt.title('Simulated Data',fontdict=font)

plt.legend()
plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# this step is used to plot a zoom-in figure
plt.xlim(240,260)
plt.ylim(3850,4250)
plt.show()
plt.tight_layout()

In [163]:
# from scipy.signal import medfilt
# basepairs_medfilt = medfilt(data['basepairs'],27)

# residuals = []
# kernal_size = np.linspace(1,101,46)
# for i in kernal_size:
#     basepairs_medfilt = medfilt(data['basepairs'],int(i))
#     residual = np.sum((basepairs_medfilt-data['basepairs'])**2)
#     residuals.append(residual)
# print(residuals)

# residual_norm = residuals/(np.max(residuals))
# plt.figure()
# plt.scatter(kernal_size,residual_norm)
# plt.xlabel("kernal size")
# plt.ylabel("normalized residuals")
# plt.title('Finding the optimal kernal size')
# # plt.xscale('log')
# plt.yscale('log')
# plt.ylim(0.1,1)
# plt.show()
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
# this is to calculate and plot the 1st deriative
from sympy import *
x = Symbol('x')
x = data['time']
y = np.array(basepairs_filter)
yprime = np.diff(y)/np.diff(x)
# print(yprime)
# print(y)
# print(np.diff(x))

plt.figure(figsize=(10,3))
plt.plot(x[:-1],yprime,color='green',linewidth=2, label='1st derivative')

plt.title('1st derivative of basepairs over time',fontdict=font)
# plt.xlim(10,50)
# plt.ylim(6400,7000)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
# this is to save the 1st derivative data for further change-point detection
txt_filename = path + "\\"+"1st_derivative_data-" + name + '.txt'
txt_data = open(txt_filename, "w")
column = np.column_stack((data['time'][:-1], yprime)) 
np.savetxt(txt_data, column,delimiter="  ")
txt_data.close()

# print(yprime)
# print(txt_filename)
print(len(y))

16008


# Now we switch to Matlab to detect the change-points
# _______________________________________________________
# After finding the steps, we analyze the change-point data

In [86]:
txtpath_changepoint = r"C:\Users\KTS260\Desktop\StepFit_Result" + "\\" + "1st_derivative_data-20200821-017-002-processedData.xlsx_properties" + '.txt'
# print(txtpath_changepoint)
data_stepfinder = pd.read_csv(txtpath_changepoint, sep=",", skipinitialspace = True)
print(data_stepfinder)
changepoint_index = np.array(data_stepfinder['IndexStep'])
# print(changepoint_index)

    IndexStep  TimeStep  LevelBefore  LevelAfter  StepSize  \
0          32        32    -0.041284   -0.024830  0.016454   
1          66        66    -0.024830   -0.011829  0.013001   
2          99        99    -0.011829   -0.004210  0.007620   
3         118       118    -0.004210    0.000370  0.004580   
4         553       553     0.000370   -0.004152 -0.004522   
..        ...       ...          ...         ...       ...   
66      15929     15929    -0.083995   -0.098007 -0.014012   
67      15946     15946    -0.098007   -0.113502 -0.015495   
68      15964     15964    -0.113502   -0.129427 -0.015924   
69      15980     15980    -0.129427   -0.145526 -0.016100   
70      15996     15996    -0.145526   -0.161630 -0.016104   

    DwellTimeStepBefore  DwellTimeStepAfter  StepError  
0                    31                  34   0.001686  
1                    34                  33   0.001331  
2                    33                  19   0.000907  
3                    19    

In [102]:
# plot the change-points on the basepair plot
plt.figure(figsize=(6,4))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.xlabel('Time/s',fontdict=font)
plt.ylabel('Basepairs',fontdict=font)
plt.plot(data['time']/1000,data['basepairs'],color='lightgray', linestyle='dashed',linewidth=1, label='Basepairs')
plt.plot(data['time']/1000,basepairs_filter,color='black',linewidth=1, label='SG_filter-windsize=65')
plt.title('Basepairs Changes as a Function of Time',fontdict=font)
plt.show()
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
# plot the change-points on the basepair plot
changepoint_time = data['time'][changepoint_index]
changepoint_basepairs = basepairs_filter[changepoint_index]
plt.scatter(changepoint_time/1000,changepoint_basepairs,marker='*',c="blue", s = 100, label = "breaking points")
# plt.xlim(300,400)
# plt.ylim(7200,7800)
plt.legend()
plt.show()

In [104]:
# define the variable of dwell_time, processivity, rate
dwell_time = np.diff(changepoint_time)
processvity = np.diff(changepoint_basepairs)
rate = np.diff(changepoint_basepairs)/np.diff(changepoint_time) *1000
rate_before_changepoint = rate[:-1]
rate_after_changepoint = rate[1:]
print(dwell_time/1000)

[  1.7     1.649   0.95   21.743   9.248 171.      4.099   2.05    7.147
   1.799  31.443   4.996   1.8     7.248   5.548   1.4     9.596  24.444
   5.447   4.75    5.398   3.098  16.195  10.798   2.5     2.098   3.55
   5.047   1.3    19.095  15.495   2.35    6.947   4.749   4.5     1.249
   7.098   6.148   2.399  12.397   7.896   6.399   2.249  18.245 216.886
   2.049   1.301   1.898   8.248   0.849   1.199   3.95    4.95    6.546
   5.549   1.45    2.898   5.65    8.396  18.796   6.247   6.149   1.199
   2.301   1.9     0.948   0.851   0.899   0.8     0.801]


In [105]:
# plot all the data
from scipy import stats
import seaborn as sns

plt.figure(figsize = (10,3))
n_bins = 10

plt.subplot(1, 4, 1)
n, bins, patches = plt.hist(dwell_time/1000, n_bins,alpha=0.5)
plt.title('Dwell Time Analysis')
plt.xlabel('Dwell time(s)')
plt.ylabel('Count')

plt.subplot(1, 4, 2)
n, bins, patches = plt.hist(processvity, n_bins,alpha=0.5)
plt.title('Processivity Analysis')
plt.xlabel('Processivity')
plt.ylabel('Count')

plt.subplot(1, 4, 3)
n, bins, patches = plt.hist(rate, n_bins,alpha=0.5)
plt.title('Rate')
plt.xlabel('Rate')
plt.ylabel('Count')

plt.subplot(1, 4, 4)
# set seaborn style
sns.set_style("white")
# Basic 2D density plot # Add thresh parameter
sns.kdeplot(rate_before_changepoint, rate_after_changepoint, cmap="Blues", shade=True, thresh=0)

plt.title('Rate Transition Analysis')
plt.xlabel('Rate Before change-point')
plt.ylabel('Rate After Change-point')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\kts260\appdata\local\programs\python\python38\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [106]:
# Save all the analyzed data in an excel file
excel_filename = path + "-filteredData" + '.xlsx'
# no idea why it causes errors without this step
writer = pd.ExcelWriter(excel_filename)
data = {'time':data['time']/1000,
        'basepairs_fileter':basepairs_filter }
df = pd.DataFrame(data)
df.to_excel(writer)
writer.save()

In [107]:
# Save all the analyzed data in an excel file
excel_filename = path + "-yprimeData" + '.xlsx'
# no idea why it causes errors without this step
writer = pd.ExcelWriter(excel_filename)
data = {'time':x[:-1],
        'yprime':yprime}
df = pd.DataFrame(data)
df.to_excel(writer)
writer.save()